In [1]:
from brainlit.utils.session import NeuroglancerSession
from brainlit.utils.swc import graph_to_paths
import napari

c:\users\shrey\anaconda3\envs\ndd-windows\lib\site-packages\python_jsonschema_objects\__init__.py:53: UserWarning: Schema version http://json-schema.org/draft-04/schema not recognized. Some keywords and features may not be supported.
  self.schema["$schema"]


# Downloading Brain data tutorial
We have prepared 2 brain volumes, as well as axon segment labels, at the below s3 urls (see `uploading_brains.ipynb`).
The method demonstrated below pulls a region of the volume around an annotated axon point set by the user.

## 1) Define Variables
- `mip` ranges from higher resolution (0) to lower resolution (1).
- `v_id` are vertex ids ranging from the soma (0) to the end of the axon (1649).
- `radius` is the radius to pull around the selected point, in voxels.

In [5]:
dir = "s3://open-neurodata/benchmarking_data/Images"
dir_segments = "s3://open-neurodata/benchmarking_data/Manual-GT"
#dir = "s3://open-neurodata/brainlit/brain1"
#dir_segments = "s3://open-neurodata/brainlit/brain1_segments"
dir_2 = "s3://open-neurodata/brainlit/brain2"
dir_2_segments = "s3://open-neurodata/brainlit/brain2_segments"
mip = 0
v_id = 0
radius = 75

In [11]:
import boto3

s3 = boto3.client('s3')
s3.download_file('open_neurodata', 'benchmarking_data', 'Images')

NoCredentialsError: Unable to locate credentials

## 2) Create a NeuroglancerSession instance and download the volume.

In [7]:
# get image and center point
ngl_sess = NeuroglancerSession(mip = mip, url = dir, url_segments=dir_segments)
img, bbox, vox = ngl_sess.pull_voxel(2, v_id, radius)
print(f"\n\nDownloaded volume is of shape {img.shape}, with total intensity {sum(sum(sum(img)))}.")

Downloading: 100%|████████████████████████████████████| 1/1 [00:00<00:00, 27.26it/s]


SkeletonDecodeError: File(s) do not exist: skeletons/2

## 3) Generate a graph from the segment data within the volume, and convert it to paths.

In [ ]:
G_sub = ngl_sess.get_segments(2, bbox)
paths = graph_to_paths(G_sub)
print(f"Selected volume contains {G_sub.number_of_nodes()} nodes and {len(paths)} paths")

## 4) View the volume with paths overlaid via napari.

In [ ]:
with napari.gui_qt():
    viewer = napari.Viewer(ndisplay=3)
    viewer.add_image(img)
    viewer.add_shapes(data=paths, shape_type='path', edge_width=0.1, edge_color='blue', opacity=0.1)
    viewer.add_points(vox, size=1, opacity=0.5)